# **Data Preprocessing**

### *1. Importing libraries and data*

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/2.\ Career\ Development/Data\ Science/4.\ Data\ Science\ Projects/Predicting\ Housing\ Prices/Data

Mounted at /content/drive
/content/drive/My Drive/2. Career Development/Data Science/4. Data Science Projects/Predicting Housing Prices/Data


In [ ]:
data = pd.read_csv('data.csv')

____________

### *2. Correcting data types*
* Some data (eg ID) is stored in the df as numeric but should be a category. 
* To correct this the respective data fields are cast to object.

In [ ]:
convert_dict = {'Id': 'object',
               'MSSubClass': 'object',
               'OverallQual': 'object',
               'OverallCond': 'object',
               'YearBuilt': 'object',
               'YearRemodAdd': 'object',
               'GarageYrBlt': 'object',
               'YrSold': 'object',
               'MoSold': 'object'
              }

data = data.astype(convert_dict)

In [ ]:
#Convert pd df to numpy array
data_values = data.values

____________

### *3. Missing data*

* Both in the numeric and categorical columns values are missing.
* Missing categorical values will be replaced with the string constant '**missing_values**'
* Missing numeric values will be replaced by the **mean**.

**3.1 Missing Values per Column**

In [ ]:
pd.set_option('display.max_rows', 81)
missing_records= data.isnull().sum().sort_values(ascending = False)
cols_with_missing = missing_records[missing_records > 0]
cols_with_missing

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

**3.2 Replacing Missing Values**

In [ ]:
# Replacing categorical values with string constant
missing_categorical=[data.columns.get_loc(index) for index, value in cols_with_missing.iteritems() if str(data[index].dtypes) == 'object']
imputer_string = SimpleImputer(missing_values = np.nan, strategy = 'constant')
imputer_string.fit(data_values[:,missing_categorical])
data_values[:,missing_categorical] = imputer_string.transform(data_values[:,missing_categorical])

In [ ]:
# Replacing numeric values with mean
missing_numeric=[data.columns.get_loc(index) for index, value in cols_with_missing.iteritems() if str(data[index].dtypes) == 'float64']
imputer_numeric = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer_numeric.fit(data_values[:,missing_numeric])
data_values[:,missing_numeric] = imputer_numeric.transform(data_values[:,missing_numeric])

In [ ]:
data_v2 = pd.DataFrame(data = data_values, columns = data.columns)

**3.3 Checking for remaining missing values**

In [ ]:
pd.set_option('display.max_rows', 81)
missing_records= data_v2.isnull().sum().sort_values(ascending = False)
cols_with_missing = missing_records[missing_records > 0]
cols_with_missing

Series([], dtype: int64)

__________

### *4. Encoding Categorical Variables*

**4.1 Identifying Categorical Variables**
* After reinserting np array into a pd df all data types are 'object'.
* To fix this the data types are corrected in the below step.

In [ ]:
int_types = [index for index, value in data.dtypes.iteritems() if str(data[index].dtypes)  == 'int64']
for i in int_types:
  convert_dict = {i: 'int64'}
  data_v2 = data_v2.astype(convert_dict)

float_types = [index for index, value in data.dtypes.iteritems() if str(data[index].dtypes)  == 'float64']
for i in float_types:
  convert_dict = {i: 'float64'}
  data_v2 = data_v2.astype(convert_dict)

object_dtypes = [index for index, value in data_v2.dtypes.iteritems() if str(data_v2[index].dtypes) == 'object']
for i in object_dtypes:
  convert_dict = {i: 'string'}
  data_v2 = data_v2.astype(convert_dict)

**4.2 Encoding Categorical Variables**
* Encoding categorical variables to dummy variables using one hot encoding.

In [ ]:
index_categorical = [index for index, val in data_v2.dtypes.iteritems() if str(val) == 'string'][1:]
enc = OneHotEncoder(handle_unknown = 'ignore')
data_v3 = data_v2

for i in index_categorical:
  col_names = [i+j for j in data_v2[i].unique()]
  col_names.sort()
  enc_df = pd.DataFrame(enc.fit_transform(data_v2[[i]]).toarray(), columns = col_names)
  data_v3 = data_v3.join(enc_df)
  del data_v3[i]

In [ ]:
data_v3[data_v3.columns[30:]].head()

,MSSubClass120,MSSubClass160,MSSubClass180,MSSubClass190,MSSubClass20,MSSubClass30,MSSubClass40,MSSubClass45,MSSubClass50,MSSubClass60,MSSubClass70,MSSubClass75,MSSubClass80,MSSubClass85,MSSubClass90,MSZoningC (all),MSZoningFV,MSZoningRH,MSZoningRL,MSZoningRM,StreetGrvl,StreetPave,AlleyGrvl,AlleyPave,Alleymissing_value,LotShapeIR1,LotShapeIR2,LotShapeIR3,LotShapeReg,LandContourBnk,LandContourHLS,LandContourLow,LandContourLvl,UtilitiesAllPub,UtilitiesNoSeWa,LotConfigCorner,LotConfigCulDSac,LotConfigFR2,LotConfigFR3,LotConfigInside,...,FenceMnPrv,FenceMnWw,Fencemissing_value,MiscFeatureGar2,MiscFeatureOthr,MiscFeatureShed,MiscFeatureTenC,MiscFeaturemissing_value,MoSold1,MoSold10,MoSold11,MoSold12,MoSold2,MoSold3,MoSold4,MoSold5,MoSold6,MoSold7,MoSold8,MoSold9,YrSold2006,YrSold2007,YrSold2008,YrSold2009,YrSold2010,SaleTypeCOD,SaleTypeCWD,SaleTypeCon,SaleTypeConLD,SaleTypeConLI,SaleTypeConLw,SaleTypeNew,SaleTypeOth,SaleTypeWD,SaleConditionAbnorml,SaleConditionAdjLand,SaleConditionAlloca,SaleConditionFamily,SaleConditionNormal,SaleConditionPartial
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


_______________

### *5. Feature Scaling*



**5.1 Feature Scaling**
*   Scaling numeric values (located in cols 0-19) using standardization.

In [ ]:
sc = StandardScaler()
scaled = sc.fit_transform(data_v3.values[:, :30])
data_v4 = pd.DataFrame(data = scaled, columns = data_v3.columns[:30])

In [ ]:
data_v4.head()

,Id,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,-1.730865,-0.229372,-0.207142,0.511418,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.912210,-0.951226,0.311725,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.347273
1,-1.728492,0.451936,-0.091886,-0.574410,1.171992,-0.288653,-0.641228,0.466465,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.318683,0.600495,0.311725,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.007288
2,-1.726120,-0.093110,0.073480,0.323060,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.311725,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.536154
3,-1.723747,-0.456474,-0.096897,-0.574410,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.296763,0.600495,1.650307,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-0.515281
4,-1.721374,0.633618,0.375148,1.364570,0.463568,-0.288653,-0.174865,0.199680,-0.045611,1.617877,-0.120242,1.299326,1.107810,-0.241061,0.789741,1.227585,1.390023,-0.211454,1.527656,0.600495,1.650307,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.869843


**5.2 Joining Scaled DF to Base DF**
* Joining the scaled numeric df with the one hot encoded categorical columns of data_v3.

In [ ]:
data_v5 = data_v4.join(data_v3[data_v3.columns[30:]])

_____________

### *6. Exporting Data Sets*

**6.1 Unscaled Data Set**

In [ ]:
%cd /content/drive/My\ Drive/2.\ Career\ Development/Data\ Science/4.\ Data\ Science\ Projects/Predicting\ Housing\ Prices/

/content/drive/My Drive/2. Career Development/Data Science/4. Data Science Projects/Predicting Housing Prices


In [ ]:
data_v3.to_csv('predicting_housing_prices_data_unscaled.csv')

**6.2 Scaled Data Set**

In [ ]:
data_v5.to_csv('predicting_housing_prices_data_scaled.csv')